<a href="https://colab.research.google.com/github/shashwatverma81/Python_RL/blob/main/EXP9_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Play BlackJack function
The player is prompted to "hit" (take another card) or "stand" (keep the current hand) until their hand's total is less than 21.

In [2]:
import random

def get_card():
    """Get a random card value."""
    card = random.randint(1, 11)  # Ace can be 1 or 11
    return card

def play_blackjack():
    """Play one round of Blackjack."""
    player_hand = [get_card(), get_card()]
    dealer_hand = [get_card(), get_card()]

    while sum(player_hand) < 21:
        action = input("Do you want to 'hit' or 'stand'? ").lower()

        if action == 'hit':
            player_hand.append(get_card())
        elif action == 'stand':
            break

        print(f"Your hand: {player_hand}")

    while sum(dealer_hand) < 17:
        dealer_hand.append(get_card())

    print(f"Your hand: {player_hand}")
    print(f"Dealer's hand: {dealer_hand}")

    if sum(player_hand) > 21:
        print("Bust! You lose.")
        return -1
    elif sum(dealer_hand) > 21 or sum(player_hand) > sum(dealer_hand):
        print("You win!")
        return 1
    elif sum(player_hand) == sum(dealer_hand):
        print("It's a tie!")
        return 0
    else:
        print("You lose.")
        return -1

def monte_carlo_simulation(num_simulations):
    """Run multiple simulations to estimate the expected outcome."""
    total_wins = 0
    total_losses = 0
    total_ties = 0

    for _ in range(num_simulations):
        result = play_blackjack()

        if result == 1:
            total_wins += 1
        elif result == 0:
            total_ties += 1
        else:
            total_losses += 1

    win_percentage = (total_wins / num_simulations) * 100
    loss_percentage = (total_losses / num_simulations) * 100
    tie_percentage = (total_ties / num_simulations) * 100

    print(f"\nResults after {num_simulations} simulations:")
    print(f"Win percentage: {win_percentage}%")
    print(f"Loss percentage: {loss_percentage}%")
    print(f"Tie percentage: {tie_percentage}%")

if __name__ == "__main__":
    num_simulations = int(input("Enter the number of Monte Carlo simulations: "))
    monte_carlo_simulation(num_simulations)


Enter the number of Monte Carlo simulations: 2
Do you want to 'hit' or 'stand'? hit
Your hand: [5, 6, 4]
Do you want to 'hit' or 'stand'? stand
Your hand: [5, 6, 4]
Dealer's hand: [9, 1, 4, 8]
You win!
Do you want to 'hit' or 'stand'? stand
Your hand: [4, 7]
Dealer's hand: [8, 7, 10]
You win!

Results after 2 simulations:
Win percentage: 100.0%
Loss percentage: 0.0%
Tie percentage: 0.0%


In [3]:
import numpy as np

class MonteCarloFirstVisitAB:
    def __init__(self, gamma=0.9):
        self.states = ['A', 'B']
        self.actions = [0, 1]  # For simplicity, assuming two actions
        self.gamma = gamma
        self.q_values = {(s, a): 0.0 for s in self.states for a in self.actions}
        self.visit_counts = {(s, a): 0 for s in self.states for a in self.actions}

    def choose_action(self, state, epsilon=0.1):
        if np.random.rand() < epsilon:
            return np.random.choice(self.actions)
        else:
            return max(self.actions, key=lambda a: self.q_values[state, a])

    def update_q_values(self, episode):
        G = 0  # Return (cumulative reward)
        visited = set()  # Keep track of visited state-action pairs

        for t in reversed(range(len(episode))):
            state, action, reward = episode[t]
            sa_pair = (state, action)

            # If the state-action pair is not visited in this episode, update Q-values
            if sa_pair not in visited:
                visited.add(sa_pair)
                self.visit_counts[sa_pair] += 1
                G = self.gamma * G + reward
                self.q_values[sa_pair] += (1 / self.visit_counts[sa_pair]) * (G - self.q_values[sa_pair])

    def run_episodes(self, num_episodes):
        for episode in range(num_episodes):
            state = np.random.choice(self.states)
            episode_history = []

            while state in self.states:
                action = self.choose_action(state)
                new_state, reward = self.take_action(state, action)
                episode_history.append((state, action, reward))

                if new_state is None:
                    break  # Terminal state reached

                state = new_state

            self.update_q_values(episode_history)

    def take_action(self, state, action):
        # Implement your environment's dynamics here
        # Return the new state and the reward
        # For simplicity, let's assume a simple environment with states A and B
        if state == 'A':
            if action == 0:
                return 'B', 1.0
            else:
                return 'A', 0.0
        elif state == 'B':
            if action == 0:
                return 'A', -1.0
            else:
                return None, 2.0  # Terminal state

# Example usage:
num_episodes = 1000

mc_first_visit_ab = MonteCarloFirstVisitAB()
mc_first_visit_ab.run_episodes(num_episodes)

# Print the learned Q-values for states A and B:
print("Learned Q-values for state A:")
for action in mc_first_visit_ab.actions:
    print(f"Q(A, {action}): {mc_first_visit_ab.q_values[('A', action)]}")

print("Learned Q-values for state B:")
for action in mc_first_visit_ab.actions:
    print(f"Q(B, {action}): {mc_first_visit_ab.q_values[('B', action)]}")


Learned Q-values for state A:
Q(A, 0): 2.8
Q(A, 1): 2.52
Learned Q-values for state B:
Q(B, 0): 1.512125
Q(B, 1): 2.0


In [4]:
import numpy as np

class MonteCarloEveryVisitAB:
    def __init__(self, gamma=0.9):
        self.states = ['A', 'B']
        self.actions = [0, 1]  # For simplicity, assuming two actions
        self.gamma = gamma
        self.q_values = {(s, a): 0.0 for s in self.states for a in self.actions}
        self.visit_counts = {(s, a): 0 for s in self.states for a in self.actions}

    def choose_action(self, state, epsilon=0.1):
        if np.random.rand() < epsilon:
            return np.random.choice(self.actions)
        else:
            return max(self.actions, key=lambda a: self.q_values[state, a])

    def update_q_values(self, episode):
        G = 0  # Return (cumulative reward)

        for t in reversed(range(len(episode))):
            state, action, reward = episode[t]
            sa_pair = (state, action)

            # Increment visit count for the state-action pair
            self.visit_counts[sa_pair] += 1

            # Update Q-value using incremental formula
            G = self.gamma * G + reward
            self.q_values[sa_pair] += (1 / self.visit_counts[sa_pair]) * (G - self.q_values[sa_pair])

    def run_episodes(self, num_episodes):
        for episode in range(num_episodes):
            state = np.random.choice(self.states)
            episode_history = []

            while state in self.states:
                action = self.choose_action(state)
                new_state, reward = self.take_action(state, action)
                episode_history.append((state, action, reward))

                if new_state is None:
                    break  # Terminal state reached

                state = new_state

            self.update_q_values(episode_history)

    def take_action(self, state, action):
        # Implement your environment's dynamics here
        # Return the new state and the reward
        # For simplicity, let's assume a simple environment with states A and B
        if state == 'A':
            if action == 0:
                return 'B', 1.0
            else:
                return 'A', 0.0
        elif state == 'B':
            if action == 0:
                return 'A', -1.0
            else:
                return None, 2.0  # Terminal state

# Example usage:
num_episodes = 1000

mc_every_visit_ab = MonteCarloEveryVisitAB()
mc_every_visit_ab.run_episodes(num_episodes)

# Print the learned Q-values for states A and B:
print("Learned Q-values for state A:")
for action in mc_every_visit_ab.actions:
    print(f"Q(A, {action}): {mc_every_visit_ab.q_values[('A', action)]}")

print("Learned Q-values for state B:")
for action in mc_every_visit_ab.actions:
    print(f"Q(B, {action}): {mc_every_visit_ab.q_values[('B', action)]}")


Learned Q-values for state A:
Q(A, 0): 2.6871295942929816
Q(A, 1): 1.4998987904887817
Learned Q-values for state B:
Q(B, 0): 0.8240793591459981
Q(B, 1): 2.0


In [ ]:
import numpy as np

def monte_carlo_first_visit(env, policy, num_episodes):
    """
    Implements the Monte Carlo first visit method.

    Args:
        env: The environment.
        policy: The policy.
        num_episodes: The number of episodes to run.

    Returns:
        The state-value function.
    """

    # Initialize the state-value function.
    state_value = np.zeros(env.n_states)

    # Run the Monte Carlo algorithm for the specified number of episodes.
    for _ in range(num_episodes):
        # Generate an episode.
        episode = generate_episode(env, policy)

        # Keep track of visited states in this episode.
        visited_states = set()

        # Update the state-value function for each state visited in the episode (first visit).
        for t in range(len(episode)):
            state, reward, _, _ = episode[t]

            # Check if it's the first visit to the state in this episode.
            if state not in visited_states:
                visited_states.add(state)
                state_value[state] = (state_value[state] * state_value[state] + reward) / (state_value[state] + 1)

    print("Learned State-Value Function (Monte Carlo First Visit):", state_value)
    return state_value

def monte_carlo_every_visit(env, policy, num_episodes):
    """
    Implements the Monte Carlo every visit method.

    Args:
        env: The environment.
        policy: The policy.
        num_episodes: The number of episodes to run.

    Returns:
        The state-value function.
    """

    # Initialize the state-value function.
    state_value = np.zeros(env.n_states)

    # Run the Monte Carlo algorithm for the specified number of episodes.
    for _ in range(num_episodes):
        # Generate an episode.
        episode = generate_episode(env, policy)

        # Update the state-value function for each state visited in the episode (every visit).
        for t in range(len(episode)):
            state, reward, _, _ = episode[t]
            state_value[state] += reward

    print("Learned State-Value Function (Monte Carlo Every Visit):", state_value)
    return state_value

def generate_episode(env, policy):
    """
    Generates an episode.

    Args:
        env: The environment.
        policy: The policy.

    Returns:
        A list of tuples representing the episode.
    """

    episode = []
    state = env.reset()
    done = False
    while not done:
        action = policy(state)
        next_state, reward, done, _ = env.step(action)
        episode.append((state, reward, next_state, done))
        state = next_state

    return episode

# Example usage with a dummy environment and policy function
class DummyEnv:
    def __init__(self):
        self.n_states = 3

    def reset(self):
        return 0

    def step(self, action):
        if action == 0:
            return 1, 1.0, False, {}
        elif action == 1:
            return 2, 0.5, False, {}

# Dummy policy that always returns action 0
def dummy_policy(state):
    return 0

# Example usage:
num_episodes = 1000

# First Visit
monte_carlo_first_visit_result = monte_carlo_first_visit(DummyEnv(), dummy_policy, num_episodes)

# Every Visit
monte_carlo_every_visit_result = monte_carlo_every_visit(DummyEnv(), dummy_policy, num_episodes)
